# Pair Programming 30/05 Test estadísticos 4 - Tatiana y Guada

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.multivariate.manova import MANOVA
from sklearn.preprocessing import StandardScaler

plt.rcParams["figure.figsize"] = (10,8) 

# Ejercicio 1
-Estandarizar las columnas de las variables predictoras


In [3]:
df = pd.read_csv("listings_limpio_2.csv", index_col=0)
df.head()

,neighbourhood_group,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm
0,Chamartín,Private room,3.021213,1,81,0.570000,1,149,3
1,Latina,Private room,2.661819,4,33,0.380000,2,131,0
2,Arganzuela,Entire home/apt,2.932182,15,0,1.347131,6,311,0
3,Centro,Entire home/apt,3.249172,5,14,0.160000,1,167,4
4,Arganzuela,Private room,2.622480,2,154,1.090000,1,344,5


In [4]:
#Estas son nuestras columnas que queremos estandarizar
df_estandarizar = df[["minimum_nights", "number_of_reviews", "reviews_per_month", "calculated_host_listings_count", "availability_365", "number_of_reviews_ltm"]]

In [5]:
st = StandardScaler()
df_std = pd.DataFrame(st.fit_transform(df_estandarizar), columns=df_estandarizar.columns)
df_std.head()

,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm
0,-0.176920,0.647995,-0.561126,-0.382089,0.006434,-0.269185
1,-0.094650,-0.047665,-0.698318,-0.346261,-0.122452,-0.493411
2,0.207007,-0.525932,0.000012,-0.202945,1.166410,-0.493411
3,-0.067227,-0.323031,-0.857171,-0.382089,0.135321,-0.194442
4,-0.149497,1.705977,-0.185653,-0.382089,1.402701,-0.119700


In [6]:
columnas = df_std.columns
df[columnas] = df_std
df.head()

,neighbourhood_group,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm
0,Chamartín,Private room,3.021213,-0.176920,0.647995,-0.561126,-0.382089,0.006434,-0.269185
1,Latina,Private room,2.661819,-0.094650,-0.047665,-0.698318,-0.346261,-0.122452,-0.493411
2,Arganzuela,Entire home/apt,2.932182,0.207007,-0.525932,0.000012,-0.202945,1.166410,-0.493411
3,Centro,Entire home/apt,3.249172,-0.067227,-0.323031,-0.857171,-0.382089,0.135321,-0.194442
4,Arganzuela,Private room,2.622480,-0.149497,1.705977,-0.185653,-0.382089,1.402701,-0.119700


# Ejercicio 2
-Cread un ANOVA, siendo la variable respuesta price y las variables predictoras el resto de las columnas del dataframe

In [7]:
# iniciamos nuestro anova. Recordemos que la variable respuesta es el precio de las casas (price)
lm = ols('price ~ minimum_nights + number_of_reviews + reviews_per_month + availability_365 + number_of_reviews_ltm', data=df).fit()
sm.stats.anova_lm(lm)

,df,sum_sq,mean_sq,F,PR(>F)
minimum_nights,1.0,0.120514,0.120514,0.654619,4.184772e-01
number_of_reviews,1.0,5.005943,5.005943,27.191645,1.863895e-07
reviews_per_month,1.0,0.047401,0.047401,0.257478,6.118646e-01
availability_365,1.0,0.097262,0.097262,0.528316,4.673258e-01
number_of_reviews_ltm,1.0,0.028692,0.028692,0.155851,6.930107e-01
Residual,17180.0,3162.813399,0.184099,NaN,NaN


👩🏻👩🏻‍🦰💬

El resultado de la columna PR(>F) se refiere al *p-valor* con el que se determina si se rechaza la hipótesis nula (recordamos también que la hipótesis nula era aquello contrario a lo que queríamos predecir), siendo ésta que la variable no tiene efecto sobre la variable respuesta. Si el valor p es menor que o igual a 0.05. Las columnas con ese valor son todas menos "reviews_per_month" y "number_of_reviews_ltm". En este caso, debemos rechazar la hipótesis nula y concluimos que esa variable si influye en el precio de las casas. 


In [8]:
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     5.758
Date:                Mon, 30 May 2022   Prob (F-statistic):           2.58e-05
Time:                        22:14:26   Log-Likelihood:                -9841.1
No. Observations:               17186   AIC:                         1.969e+04
Df Residuals:                   17180   BIC:                         1.974e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                 3.0932      0.003    945.068      0.000       3.087       3.100
minimum_nights           -0.0035      0.003     -1.077      0.282      -0.010       0.003
number_of_reviews        -0.0154      0.004     -3.664      0.000      -0.024      -0.007
reviews_per_month        -0.0014      0.005     -0.285      0.776      -0.011       0.008
availability_365          0.0025      0.003      0.758      0.448      -0.004       0.009
number_of_reviews_ltm    -0.0019      0.005     -0.395      0.693      -0.011       0.007
==============================================================================
Omnibus:                        2.701   Durbin-Watson:                   1.597
Prob(Omnibus):                  0.259   Jarque-Bera (JB):                2.626
Skew:                          -0.004   Prob(JB):                        0.269
Kurtosis:                       2.940   Cond. No.                         2.71
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

👩🏻👩🏻‍🦰💬

Interpretemos estos resultados:

    - `coef`: es el coeficiente que representa los cambios medios en la variable respuesta para una unidad de cambio en la variable predictora mientras se mantienen constantes el resto de las variables predictoras. Es la tasa de cambio, nos indica lo que cambia la variable respuesta (en nuestro caso el precio) por cada unidad que aumenta la variable predictora. En nuestros resultados, todas las columnas han salido negativas, excepto "availability_365", aunque están cerca de cero. Esto quiere decir que la única variable que cambia considerablemente al aumentar el precio sería availability_365.

    - `std err`: es el error estándar del coeficiente. Se usa para medir la precisión de la estimación del coeficiente. Cuanto menor sea el error estándar, más precisa será la estimación. En nuestro dataset, sus valores son bajos, por lo que la precisión es buena.

    - `t` : es el resultado de dividir el coeficiente entre su error estándar. Como era de esperar tras ver el coeficiente en el primer punto, estos valores son más bajos, llegando al punto de que la variable "number_of_reviews" llega a estar en más de -3.

    - `P`: Para determinar si cualquiera de las diferencias entre las medias es estadísticamente significativa, deberemos comparar el valor p con el nivel de significancia para evaluar la hipótesis nula. En "number_of_reviews" es 0, por lo que se acepta la hipótesis nula, mientras que en el resto de variable dista de ser 0, aunque en el caso de "minimum_nights" no se aleja demasiado.

    - `R-squared`: el porcentaje de la variación de la variable respuesta está explicado por nuestro modelo. Es decir, nuestros variables predictoras explican el   0.2% de la variación en los precios de las casas.